In [1]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable, Optional
from datetime import datetime, timedelta
from IPython.display import display

import numpy as np
import pandas as pd

from sesto.constants import CURRENCIES, MT5Timeframe
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.backtester import Backtester, Trade

MetaTrader 5 initialized successfully.


In [2]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 8
Number of pairs: 56


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
TIMEFRAMES = [MT5Timeframe.M5, MT5Timeframe.M15, MT5Timeframe.H1, MT5Timeframe.H4]
DATA_FROM_DATE = datetime(2024, 4, 5)
DATA_TO_DATE = datetime(2024, 9, 19)

MA_PERIODS = [7, 14, 21]

CSM_PERIOD = 21

In [5]:
BACKTEST_FROM_DATE = datetime(2024, 1, 1)
BACKTEST_TO_DATE = datetime(2024, 9, 20)
MAIN_TIMEFRAME = MT5Timeframe.H1

INITIAL_CAPITAL = 10000

CSM_TREND_LONG_THRESHOLD = 65
CSM_TREND_SHORT_THRESHOLD = 35
CSM_REVERSAL_LONG_THRESHOLD = 85
CSM_REVERSAL_SHORT_THRESHOLD = 15
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
ATR_VOLATILITY_THRESHOLD = 0.02

CAPITAL_PER_TRADE = 200  # $200 per trade

RR_MULTIPLIER = 2
TRADE_SL_MULTIPLIER = 0.0025

TRANSACTION_COST = 0.0001
SLIPPAGE = 0.0001

LEVERAGE = 500

In [6]:
mtd.fetch_data_all_timeframes(PAIRS, TIMEFRAMES, DATA_FROM_DATE, DATA_TO_DATE)

Failed to fetch data for symbol: USDEUR on timeframe: MT5Timeframe.M5
No data fetched for pair: USDEUR on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDGBP on timeframe: MT5Timeframe.M5
No data fetched for pair: USDGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: USDJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCAD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDAUD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDAUD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDNZD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDNZD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURUSD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: EURJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCAD on timeframe: MT5T

In [7]:
# Remove symbols with no data for any timeframe
for timeframe in TIMEFRAMES:
    empty_symbols = [symbol for symbol, df in mtd.data[timeframe].items() if df is None or df.empty]
    for symbol in empty_symbols:
        del mtd.data[timeframe][symbol]
        if symbol in PAIRS:
            PAIRS.remove(symbol)

print(f"Remaining pairs after removing empty data: {len(PAIRS)}")

Remaining pairs after removing empty data: 28


In [8]:
import plotly.graph_objects as go

def plot_data_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                actual_rows = len(mtd.data[timeframe][symbol])
                normalized_value = actual_rows / expected_rows[timeframe]
                heatmap_data.append({
                    'Timeframe': timeframe.name,
                    'Symbol': symbol,
                    'Normalized Row Count': normalized_value
                })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Row Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Row Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Row Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Row Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_data_availability()

In [9]:
from sesto.indicators import SMA, EMA, RSI, ROC, MACD, BB, ATR

for timeframe, pairs_data in mtd.data.items():
    for pair, df in pairs_data.items():
        if df is not None and not df.empty:
            for period in MA_PERIODS:
                SMA(df, period)
                EMA(df, period)
                ROC(df, period)
                RSI(df, period)
                BB(df, period, 2)
                ATR(df, period)
    
            df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2

display(mtd.data[MT5Timeframe.H1]['EURUSD'].head())



,time,open,high,low,close,tick_volume,spread,real_volume,sma-7,ema-7,...,bbu-14-2,bbl-14-2,sma-21,ema-21,roc-21,rsi-21,bbm-21-2,bbu-21-2,bbl-21-2,psm
0,2024-04-04 21:00:00,1.08600,1.08600,1.08466,1.08500,2883,0,0,NaN,1.085000,...,NaN,NaN,NaN,1.085000,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-04 22:00:00,1.08499,1.08507,1.08333,1.08336,3453,0,0,NaN,1.084590,...,NaN,NaN,NaN,1.084851,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-04 23:00:00,1.08336,1.08381,1.08315,1.08362,1028,0,0,NaN,1.084348,...,NaN,NaN,NaN,1.084739,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-05 00:00:00,1.08361,1.08397,1.08229,1.08373,2068,15,0,NaN,1.084193,...,NaN,NaN,NaN,1.084647,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-05 01:00:00,1.08367,1.08396,1.08352,1.08360,659,7,0,NaN,1.084045,...,NaN,NaN,NaN,1.084552,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import plotly.graph_objects as go
import pandas as pd

def plot_indicator_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                df = mtd.data[timeframe][symbol]
                if df is not None and not df.empty:
                    # Check for the presence of indicators
                    indicator_columns = [col for col in df.columns if col.startswith(('sma-', 'ema-', 'rsi-', 'roc-', 'macd', 'bb'))]
                    if indicator_columns:
                        actual_rows = df[indicator_columns].dropna().shape[0]
                        normalized_value = actual_rows / expected_rows[timeframe]
                        heatmap_data.append({
                            'Timeframe': timeframe.name,
                            'Symbol': symbol,
                            'Normalized Indicator Count': normalized_value
                        })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Indicator Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Indicator Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Indicator Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Indicator Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_indicator_availability()

In [11]:
import pandas_ta as ta

for timeframe in TIMEFRAMES:
    for currency in CURRENCIES:
        # Initialize DataFrames to hold PSM values for the current currency as base and quote
        base_psm_df = pd.DataFrame()
        quote_psm_df = pd.DataFrame()
        
        # Iterate over all pair data for the current timeframe
        for symbol, df in mtd.data[timeframe].items():
            if df is None or df.empty:
                continue
            
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency:
                # Use PSM as is for base currency
                base_psm_df[symbol] = df['psm']
                base_psm_df['time'] = df['time']
            elif currency == quote_currency:
                # Invert PSM for quote currency
                quote_psm_df[symbol] = -df['psm']
                quote_psm_df['time'] = df['time']
        
        # Calculate base CSM
        if not base_psm_df.empty:
            base_csm_df = base_psm_df.groupby('time').mean()
            base_csm_df['csm'] = base_csm_df.mean(axis=1)
            base_csm_df['csm'] = base_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = base_csm_df['csm'].min()
            max_csm = base_csm_df['csm'].max()
            base_csm_df['csm'] = 100 * (base_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for base CSM
            MACD(base_csm_df, 'csm', 12, 26, 9)
        
        # Calculate quote CSM
        if not quote_psm_df.empty:
            quote_csm_df = quote_psm_df.groupby('time').mean()
            quote_csm_df['csm'] = quote_csm_df.mean(axis=1)
            quote_csm_df['csm'] = quote_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = quote_csm_df['csm'].min()
            max_csm = quote_csm_df['csm'].max()
            quote_csm_df['csm'] = 100 * (quote_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for quote CSM
            MACD(quote_csm_df, 'csm', 12, 26, 9)
        
        # Add CSM and MACD values to all relevant symbols in mtd.data
        for symbol in mtd.data[timeframe]:
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency and not base_csm_df.empty:
                mtd.data[timeframe][symbol]['base_csm'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['csm'])
                mtd.data[timeframe][symbol]['base_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd'])
                mtd.data[timeframe][symbol]['base_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['base_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['base_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['base_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)

            elif currency == quote_currency and not quote_csm_df.empty:

                mtd.data[timeframe][symbol]['quote_csm'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['csm'])
                mtd.data[timeframe][symbol]['quote_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd'])
                mtd.data[timeframe][symbol]['quote_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['quote_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['quote_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['quote_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)



In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

def plot_random_data():
    # Choose a random timeframe and symbol
    random_timeframe = random.choice(TIMEFRAMES)
    random_symbol = random.choice(PAIRS)

    # Get the data for the chosen timeframe and symbol
    df = mtd.data[random_timeframe][random_symbol]

    # Create subplots
    fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                        subplot_titles=('Close Price + SMA', 'CSM Values', 'Base CSM MACD', 'Quote CSM MACD', 'RSI', 'ROC'))

    # 1. Close Price + SMA
    fig.add_trace(go.Scatter(x=df['time'], y=df['close'], name='Close Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['sma-14'], name='SMA-14'), row=1, col=1)

    # 2. CSM Values
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm'], name='Base CSM'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm'], name='Quote CSM'), row=2, col=1)

    # 3. Base CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd'], name='Base MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd_signal'], name='Base Signal'), row=3, col=1)
    fig.add_bar(x=df['time'], y=df['base_csm_macd_histogram'], name='Base Histogram', row=3, col=1)

    # 4. Quote CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd'], name='Quote MACD'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd_signal'], name='Quote Signal'), row=4, col=1)
    fig.add_bar(x=df['time'], y=df['quote_csm_macd_histogram'], name='Quote Histogram', row=4, col=1)

    # 5. RSI
    fig.add_trace(go.Scatter(x=df['time'], y=df['rsi-14'], name='RSI-14'), row=5, col=1)

    # 6. ROC
    fig.add_trace(go.Scatter(x=df['time'], y=df['roc-14'], name='ROC-14'), row=6, col=1)

    # Update layout
    fig.update_layout(
        height=1500, 
        width=1600, 
        title_text=f"Chart for {random_symbol} ({random_timeframe.name})", 
        template='plotly_dark'
    )

    # Update x-axis to include range slider
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        row=6, col=1  # Apply to the bottom subplot
    )

    # Hide rangeslider for all subplots except the bottom one
    for i in range(1, 6):
        fig.update_xaxes(rangeslider_visible=False, row=i, col=1)

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="CSM", row=2, col=1)
    fig.update_yaxes(title_text="Base MACD", row=3, col=1)
    fig.update_yaxes(title_text="Quote MACD", row=4, col=1)
    fig.update_yaxes(title_text="RSI", row=5, col=1)
    fig.update_yaxes(title_text="ROC", row=6, col=1)

    # Show the plot
    fig.show()

# plot_random_data()

In [13]:
class MyStrategy(Backtester):    
    def entry_condition(
        self, 
        symbol: str, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe,
    ) -> Optional[Dict]:
        # Check if we already have an open trade for this symbol
        if any(trade.symbol == symbol for trade in open_trades):
            return None

        base_csm = row.get('base_csm')
        quote_csm = row.get('quote_csm')
    
        trend_up = (
            base_csm > CSM_TREND_LONG_THRESHOLD and 
            quote_csm < CSM_TREND_SHORT_THRESHOLD
        )

        trend_down = (
            base_csm < CSM_TREND_SHORT_THRESHOLD and 
            quote_csm > CSM_TREND_LONG_THRESHOLD
        )

        entry = None

        if trend_up:
            entry_price = row['close'] * ( 1 + SLIPPAGE)

            entry = {
                'entry_price': entry_price,
                'type': 'long',
                'capital_allocation': CAPITAL_PER_TRADE,
                'tp': entry_price * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                'sl': entry_price * (1 - TRADE_SL_MULTIPLIER)
            }
        elif trend_down:
            entry_price = row['close'] * ( 1 - SLIPPAGE)

            entry = {
                'entry_price': entry_price,
                'type': 'short',
                'capital_allocation': CAPITAL_PER_TRADE,
                'tp': entry_price * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                'sl': entry_price * (1 + TRADE_SL_MULTIPLIER)
            }

        # Ensure no open trades for the symbol
        if entry and not any(trade.symbol == symbol for trade in open_trades):
            return entry

        return None

    def exit_condition(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ) -> bool:
        return False

    def trailing_stop(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ):
        """
        Implement a trailing stop that adjusts the stop loss to make the trade risk-free
        once the unrealized PnL exceeds 5% of the capital allocation.
        """
        # Define the threshold for trailing stop activation
        TRAILING_PNL_THRESHOLD = 0.05 * trade.used_capital  # 5% of capital allocation

        # Check if unrealized PnL exceeds the threshold
        if trade.unrealized_pnl > TRAILING_PNL_THRESHOLD:
            # Check if trailing stop has not been set yet
            if trade.potential_loss_usd > 0 or trade.potential_loss_percent > 0:
                old_metrics = {
                    'sl': trade.sl,
                    'potential_loss_usd': trade.potential_loss_usd,
                    'potential_loss_percent': trade.potential_loss_percent,
                    'potential_profit_usd': trade.potential_profit_usd,
                    'potential_profit_percent': trade.potential_profit_percent
                }

                # Calculate the risk-free stop loss price
                if trade.type == 'long':
                    # For long trades, stop loss is set to entry price accounting for slippage and transaction cost
                    trade_break_even_price = trade.entry_price * (1 + self.slippage) * (1 + self.transaction_cost)
                else:
                    # For short trades, stop loss is set to entry price accounting for slippage and transaction cost
                    trade_break_even_price = trade.entry_price * (1 - self.slippage) * (1 - self.transaction_cost)

                # Update the trade's stop loss
                trade.sl = trade_break_even_price

                # Update potential loss to reflect the new stop loss (should be zero)
                trade.calculate_potential_outcomes()

                new_metrics = {
                    'sl': trade.sl,
                    'potential_loss_usd': trade.potential_loss_usd,
                    'potential_loss_percent': trade.potential_loss_percent,
                    'potential_profit_usd': trade.potential_profit_usd,
                    'potential_profit_percent': trade.potential_profit_percent
                }

                # Create DataFrame to compare old and new metrics
                comparison_df = pd.DataFrame({
                    'Metric': old_metrics.keys(),
                    'Old Value': old_metrics.values(),
                    'New Value': new_metrics.values()
                })

                print(f"{row['time']} : TRAILING STOP - {trade.symbol} - UNREALIZED PNL: ${trade.unrealized_pnl:.3f} - NEW SL: ${trade.sl:.3f}")
                # display(comparison_df)

        # Additional condition: If unrealized PnL > 5% but potential losses are already zero,
        # reset TP and SL to secure profits
        if trade.unrealized_pnl > TRAILING_PNL_THRESHOLD and \
           trade.potential_loss_usd == 0 and trade.potential_loss_percent == 0:
            if trade.type == 'long':
                # Update take profit and stop loss
                # trade.tp = row['close'] * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER))
                trade.sl = row['close'] * (1 - TRADE_SL_MULTIPLIER)
                print(f"Updated TP to {trade.tp:.5f} and SL to {trade.sl:.5f} for long trade on {trade.symbol}.")
            else:
                # Update take profit and stop loss for short positions
                # trade.tp = row['close'] * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER))
                trade.sl = row['close'] * (1 + TRADE_SL_MULTIPLIER)
                print(f"Updated TP to {trade.tp:.5f} and SL to {trade.sl:.5f} for short trade on {trade.symbol}.")

            # Recalculate potential loss based on new SL
            trade.calculate_potential_outcomes()


In [14]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    main_timeframe=MAIN_TIMEFRAME,
    transaction_cost=TRANSACTION_COST, 
    slippage=SLIPPAGE,
    leverage=LEVERAGE
)

backtest.run()

2024-04-25 09:00:00 - OPENED TRADE - long - USDJPY - ENTRY: $155.679 - TP: $156.457 - SL: $155.289 - LIQ: $155.367
2024-04-26 10:00:00 - CLOSED TRADE - long - USDJPY - CLOSE: $156.45695913149999 - PNL: $469.95 - SL: $155.710 - TP: $156.457 - REASON: tp
2024-05-02 11:00:00 - OPENED TRADE - short - USDJPY - ENTRY: $155.127 - TP: $154.352 - SL: $155.515 - LIQ: $155.438
2024-05-02 17:00:00 - CLOSED TRADE - short - USDJPY - CLOSE: $154.3518482715 - PNL: $470.05 - SL: $155.096 - TP: $154.352 - REASON: tp
2024-05-02 17:00:00 - OPENED TRADE - short - USDJPY - ENTRY: $154.051 - TP: $153.280 - SL: $154.436 - LIQ: $154.359
2024-05-02 21:00:00 - CLOSED TRADE - short - USDJPY - CLOSE: $153.280340433 - PNL: $470.05 - SL: $154.020 - TP: $153.280 - REASON: tp
2024-05-02 21:00:00 - OPENED TRADE - short - USDJPY - ENTRY: $153.039 - TP: $152.274 - SL: $153.421 - LIQ: $153.345
2024-05-02 23:00:00 - CLOSED TRADE - short - USDJPY - CLOSE: $153.42129133650002 - PNL: $-280.02 - SL: $153.421 - TP: $152.274 - R

In [15]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log[['type', 'entry_price', 'tp', 'sl', 'liquidation_price','pnl']]
# trade_log.to_csv('./log.csv')

,entry_price,type,tp,sl,liquidation_price,pnl
0,155.678566,long,156.456959,155.709704,155.367209,469.950000
1,155.127486,short,154.351848,155.096462,155.437741,470.050000
2,154.050593,short,153.280340,154.019785,154.358695,470.050000
3,153.038695,short,152.273501,153.421291,153.344772,-280.025000
4,153.681630,short,152.913222,153.650896,153.988994,470.050000
...,...,...,...,...,...,...
2137,104.155414,long,104.676192,104.176247,103.947104,247.044302
2138,95.919591,long,96.399189,95.938776,95.727752,80.924534
2139,0.570567,long,0.573420,0.570681,0.569426,114.219107
2140,88.194819,long,88.635793,87.974332,88.018429,-64.940282


In [16]:
performance_report = backtest.generate_report()
performance_report

KeyError: 'position_type'